In [1]:
import pandas as pd
import tweepy
import time
import datetime
from collections import deque

In [29]:
import pandas as pd
import tweepy
import time
import datetime
from collections import deque


class TwitterMiner:

    def __init__(self, keys, keyword, last=500, once=1000):
        self.keys = keys
        self.keyword = keyword
        self.last_tweets = deque(maxlen = last)
        self.tweets_at_once = once
        auth = tweepy.OAuthHandler(self.keys['consumer_key'], self.keys['consumer_secret'])
        auth.set_access_token(self.keys['access_token_key'], self.keys['access_token_secret'])
        self.api = tweepy.API(auth, wait_on_rate_limit_notify=True, wait_on_rate_limit=True)

    def miner(self):
        data = []
        try:
            results = tweepy.Cursor(self.api.search, q=self.keyword, lang='en', tweet_mode='extended').items(
                self.tweets_at_once)
        except tweepy.TweepError:
            time.sleep(901)
            results = tweepy.Cursor(self.api.search, q=self.keyword, lang='en', tweet_mode='extended').items(
                self.tweets_at_once)

        for tweet in results:
            if tweet.id in self.last_tweets:
                continue
            self.last_tweets.append(tweet.id)
            hashtags = [tweet.entities["hashtags"][i]['text'] for i in range(0, len(tweet.entities["hashtags"]))]
            mined = {
                'tweet_id': tweet.id,
                'mined_at': str(datetime.datetime.now()),
                'name': tweet.user.name,
                'screen_name': tweet.user.screen_name,
                'followers_count': tweet.user.followers_count,
                'friends_count': tweet.user.friends_count,
                'user_location': tweet.user.location,
                'date': str(tweet.created_at),
                'text': tweet.full_text,
                'hashtags': hashtags,
                'retweet_count': tweet.retweet_count,
                'favorite_count': tweet.favorite_count,
                'device': tweet.source,
                'location': tweet.place
            }

            try:
                mined['retweet_text'] = tweet.retweeted_status.full_text
            except:
                mined['retweet_text'] = 'None'
            try:
                mined['quote_text'] = tweet.quoted_status.full_text
                mined['quote_screen_name'] = tweet.quoted_status.user.screen_name
            except:
                mined['quote_text'] = 'None'
                mined['quote_screen_name'] = 'None'
            try:
                mined['media'] = tweet.entities.media.media_url
            except:
                mined['media'] = 'None'
            try:
                mined['truncated_text'] = tweet.truncated
            except:
                mined['truncated_text'] = 'None'

            data.append(mined)

        return data

    def boss(self, wait=900, batch=96):
        while True:
            counter = 0
            twitter_data = {}
            while counter < batch:
                name = 'batch number ' + str(counter)
                print(name)
                twitter_data[name] = []
                twitter_data[name].append(self.miner())
                time.sleep(wait)
                counter += 1

            all_tweets = pd.concat([pd.DataFrame(twitter_data[i][0]) for i in twitter_data]).reset_index(drop=True)
            filename = 'tweets_about_' + self.keyword + '_' + str(datetime.date.today()) + '.csv'
            all_tweets.to_csv(filename)


In [30]:
twitter_keys = {
    'consumer_key':        'g3OU7SzfpnE2AoPzWzC7My8O9',
    'consumer_secret':     'kIOJAjk4Zun00BvCNCgK4NCSq2X5suMb88IJ12UcSzQUhVLiie',
    'access_token_key':    '953693483012902912-fXIsILeot5yu6LLk7BBxUiEfi4DwrQD',
    'access_token_secret': 'vNhUqPByb1SZQMIVuGwhJX8BaGAkROiEMVQ3N6JyYEB2l'
}

In [31]:
miner = TwitterMiner(twitter_keys, '#brexit')

In [12]:
%time data = miner.miner()

CPU times: user 2.36 s, sys: 123 ms, total: 2.48 s
Wall time: 53.4 s


In [7]:
1187353343435464705 in miner.last_tweets

True

In [92]:
counter = 0
twitter_data = {}
wait = 1

In [93]:
while counter <= 10:
    name = 'batch number ' + str(counter)
    twitter_data[name] = []
    twitter_data[name].append(miner.miner())
    time.sleep(wait)
    counter += 1

In [94]:
all_tweets = pd.concat([pd.DataFrame(twitter_data[i][0]) for i in twitter_data]).reset_index(drop = True)

In [57]:
with pd.option_context('display.max_rows', 200, 'display.max_columns', None):  # more options can be specified also
    print(all_tweets)

                tweet_id                    mined_at  \
0    1187333217080221696  2019-10-24 13:41:30.084116   
1    1187333215847145472  2019-10-24 13:41:30.084131   
2    1187333215742255108  2019-10-24 13:41:30.084136   
3    1187333203134164992  2019-10-24 13:41:30.084141   
4    1187333193655033857  2019-10-24 13:41:30.084145   
5    1187333188873543681  2019-10-24 13:41:30.084150   
6    1187333174264815616  2019-10-24 13:41:30.084154   
7    1187333170917761024  2019-10-24 13:41:30.084159   
8    1187333162659172352  2019-10-24 13:41:30.084163   
9    1187333160910098433  2019-10-24 13:41:30.084167   
10   1187333217080221696  2019-10-24 13:41:31.799618   
11   1187333215847145472  2019-10-24 13:41:31.799683   
12   1187333215742255108  2019-10-24 13:41:31.799710   
13   1187333203134164992  2019-10-24 13:41:31.799735   
14   1187333193655033857  2019-10-24 13:41:31.799760   
15   1187333188873543681  2019-10-24 13:41:31.799782   
16   1187333174264815616  2019-10-24 13:41:31.79

In [59]:
all_tweets.dtypes

tweet_id              int64
mined_at             object
name                 object
screen_name          object
followers_count       int64
friends_count         int64
user_location        object
date                 object
text                 object
hashtags             object
retweet_count         int64
favorite_count        int64
device               object
location             object
retweet_text         object
quote_text           object
quote_screen_name    object
media                object
truncated_text         bool
dtype: object

In [66]:
all_tweets.memory_usage(deep = True).sum()

181087

In [61]:
filename = 'tweets_about_' + '#brexit' + '_' + str(datetime.date.today()) + '.hdf'
all_tweets.to_hdf(filename, key = 'all_tweets')

/home/grzegorz/anaconda3/envs/Twitter/lib/python3.7/site-packages/pandas/core/generic.py:2530: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['mined_at', 'name', 'screen_name', 'user_location', 'date', 'text', 'hashtags', 'device', 'location', 'retweet_text', 'quote_text', 'quote_screen_name', 'media']]

  pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [70]:
filename = 'tweets_about_' + '#brexit' + '_' + str(datetime.date.today()) + '.csv'
all_tweets.to_csv(filename)

In [80]:
hashtag = [{'text': 'Brexit', 'indices': [110, 117]}]

In [95]:
all_tweets

,tweet_id,mined_at,name,screen_name,followers_count,friends_count,user_location,date,text,hashtags,retweet_count,favorite_count,device,location,retweet_text,quote_text,quote_screen_name,media,truncated_text
0,1187341469104914432,2019-10-24 14:14:20.697563,Sara Hassan,SaraHas61880960,109,220,"Corby, England",2019-10-24 12:14:00,RT @Andy_Curium: Fascinating explanation from ...,[Brexit],4,0,Twitter Web App,None,Fascinating explanation from @profsicollinson ...,None,None,None,False
1,1187341464428306432,2019-10-24 14:14:20.697580,Bart van Horck,BartvanHorck,834,1165,"Den Haag, Netherlands",2019-10-24 12:13:59,RT @antoguerrera: Oh-oh...\n\n#BREXIT: UK HOUS...,[BREXIT],1,0,Twitter for iPhone,None,Oh-oh...\n\n#BREXIT: UK HOUSE OF COMMONS LEADE...,None,None,None,False
2,1187341452017319936,2019-10-24 14:14:20.697586,VmE #66Million #COG⚙ #FBPE #RevokeA50 🇪🇺❤🇬🇧,thebluecase,6079,6323,,2019-10-24 12:13:56,RT @IrinavonWiese: I've lived in the UK over h...,[Brexit],514,0,Twitter for Android,None,I've lived in the UK over half my life. I foun...,None,None,None,False
3,1187341439946108928,2019-10-24 14:14:20.697591,Emma Valasco,XXXSexContacts,704,69,"London, England",2019-10-24 12:13:53,@andywigmore @jsan1972 It is to be hoped ~ tha...,"[Brexit, Brexit]",0,0,Twitter Web App,None,None,None,None,None,False
4,1187341425849126913,2019-10-24 14:14:20.697596,#FBPE 🇪🇺🇩🇪🇰🇪🇬🇧 #FraudMeansFraud 🕷💚,ldrscke,3285,5003,"Newark-on-Trent, England",2019-10-24 12:13:50,RT @CoachDaveBeal: The UK will not exit the EU...,[],12,0,Twitter Web App,None,The UK will not exit the EU on 31.10.19. \nI'l...,None,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,1187341518882967553,2019-10-24 14:14:38.817675,Iain Ogilvie 🇬🇧🇪🇺🎪 #FBPE #RevokeA50,TheAeronut,13920,13962,Old barn in the Cotswolds,2019-10-24 12:14:12,RT @abcpoppins: Speaker hasn’t chosen our @Lib...,[Brexit],10,0,Twitter Web App,None,Speaker hasn’t chosen our @LibDems amendment f...,None,None,None,False
106,1187341517884678145,2019-10-24 14:14:38.817702,Hugh Osborne,HughRSOsborne,1288,2311,Leafy Bucks,2019-10-24 12:14:12,RT @JackDunc1: Yet whenever Jeremy Corbyn trie...,[],5,0,Twitter for Android,None,Yet whenever Jeremy Corbyn tries to raise thes...,None,None,None,False
107,1187341516433510400,2019-10-24 14:14:38.817730,Martin Shovel,MartinShovel,14034,997,United Kingdom,2019-10-24 12:14:11,RT @Banxcartoons: From today's @FT #Brexit #Do...,"[Brexit, DowningStreetSource, DominicCummings]",4,0,Twitter for iPhone,None,From today's @FT #Brexit #DowningStreetSource ...,None,None,None,False
108,1187341515674324992,2019-10-24 14:14:38.817755,PeaNell #StopBrexit #NHS #Remainer #Green #Glaws,northlondonpoly,2123,3566,"Wiltshire, European Union",2019-10-24 12:14:11,@Notenoughlove1 @RichardBentall @Slartyb965432...,[Brexit],0,0,Twitter for Android,None,None,None,None,None,False


In [32]:
miner.boss(batch=1, wait=1)

batch number 0
batch number 0


KeyboardInterrupt: 